# Notes
1. The subjects are labelled as 0-14, while the species are from 1-10. 
(Due to a mistake I made while outputing the data into a format for Python...)

In [ ]:
# 1 import the moduls
import scipy.io as sio
import librosa as lb
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler 
from sklearn.decomposition import PCA
from IPython.display import Audio

# Spectral

In [ ]:
# 2 Function for extracting features 
def getfeatures(fdata):

    data = np.array([lb.feature.rmse(y = fdata,frame_length=1024, hop_length=512)])[0,0]
    threshold = np.ndarray.max(data[1:10])

    length = fdata.size
    data_order = data.argsort()[-5:][::-1]
    for i in range(0,4):
        start = data_order[i] * 512 - 4090
        end = data_order[i] * 512 + 4099
        if length < end + 1 :
            end = length - 1
    
        fade = fdata[end] / 2
        target = np.append(fdata[start : end],[fade])
        target = np.append(target,0)
    
        if max(np.correlate(target,target,'same')) > 10:
            break
            
         
    mfccs = lb.feature.mfcc(y=target, sr=44100, n_mfcc=14, n_fft = 8192,hop_length = 8192)    
    mfccs = mfccs[1:14]
    return mfccs, target

In [ ]:
# 3 Initiate the feature vectors
subjects = []
species_all = []
mfcc_all = []

In [ ]:
# 4 Get voice imitation in (Need to manually go from test_1 to test_6 for all imitations)
test = sio.loadmat('test_1.mat')
audio = test['audio_1']
num_row = int(np.shape(audio)[0])

# 5 And for each audio clip in each .mat, extract the features
for i in tqdm(range(num_row)):
    temp_sub = audio[i,0]
    temp_spe = audio[i,1]
    rdata = audio[i,2:-1]
    rdata_nz = np.nonzero(rdata)[0][-1]
    fdata = rdata[0:rdata_nz]
    
    mfccs,target = getfeatures(fdata)

    subjects.append(temp_sub)
    species_all.append(temp_spe)
    
    #!!!! This part need to be commented out if the code is being not runned on test_1
    if i == 0:
        mfcc_all = mfccs
    else:    
    #!!!    
        mfcc_all = np.append(mfcc_all,mfccs,1)

In [ ]:
# 6 prepare data for clustering and visualization
data = mfcc_all
species_all = np.array(species_all)
data_flip = list(map(list, zip(*data)))

In [ ]:
# 7 Do clustering(for labels) and PCA(only for visualization)
mfcc_pca = PCA(n_components = 2)
results = mfcc_pca.fit_transform(data_flip)
db = KMeans(n_clusters=2).fit(data_flip)

In [ ]:
# 8 Visualizing clustering results
fig = plt.figure(figsize=(12, 9))
plt.scatter(results[:,0], results[:,1],c=db.labels_.astype(np.float), edgecolor='k')
    
plt.show() 

In [ ]:
# 9 Calculate the numbers of whistled recordings for each species & subject
sub = []
for i in range(15):
    ind_whis = np.where(db.labels_ == 0)[0]
    sub.append(sum((subjects[s] == i) for s in ind_whis))
sub = np.array(sub)

spe = []
for j in range(1,11):
    spe_whis = np.where(db.labels_ == 0)[0]
    spe.append(sum((species_all[t] == j) for t in spe_whis))
spe = np.array(spe)  


In [ ]:
# 10 Plot whistled recording percentage of each species
fig = plt.figure(figsize=(12, 6))
subject_len = len(set(subjects))
species1 = plt.bar(range(10),subject_len*10-spe)
species2 = plt.bar(range(10),spe,bottom=(subject_len*10-spe))

plt.ylabel('numbers of recording')

plt.title('whistle as strategy')
plt.xlabel('species')
plt.xticks(np.arange(10),('Mourning dove','Sora','sparrow','Northern cardinal','Veery','Chikadee','Vireo','Prairie warbler','Yellowthroat','Blue warbler'))
plt.tick_params(axis = 'both', labelsize = 8)
plt.legend((species1[0], species2[0]), ('Whistle', 'Non-whistle'))
plt.ylim(0,(subject_len+2)*10)

plt.show()

In [ ]:
# 11 Plot whistle results between subjects
fig = plt.figure(figsize=(10, 6))
sub = sub[sub>0]
non_sub = 100-sub
subjects1 = plt.bar(range(1,subject_len+1),non_sub)
subjects2 = plt.bar(range(1,subject_len+1),sub,bottom=non_sub)

plt.ylabel('numbers of recording')
plt.xlabel('subjects')
plt.title('whistle as strategy')
plt.xticks(np.arange(1,subject_len+1))
plt.legend((subjects1[0], subjects2[0]), ('Whistle', 'Non-whistle'))
plt.ylim(0,120)

plt.show()

# Temporal

In [ ]:
# t-2 Voice Activity Detection function
def detect_activity(y, sr,
        n_mels=128, fmin=1000, fmax=11025, 
        hop_length=512, gain=0.8, bias=10, power=0.25, pcen_time_constant=0.06, eps=1e-06,
        medfilt_time_constant=None, normalized=True,
        peak_threshold=0.45, activity_threshold=0.2):
    # 1. Compute mel-frequency spectrogram
    melspec = lb.feature.melspectrogram(
        y, sr=sr, fmin=fmin, fmax=fmax, hop_length=hop_length,
        n_mels=n_mels)
    
    # 2. Compute per-channel energy normalization (PCEN-SNR)
    pcen = lb.core.pcen(melspec, sr=sr, gain=gain, bias=bias,
        power=power, hop_length=hop_length,
        time_constant=pcen_time_constant, eps=eps)
    
    # 3. compute PCEN-SNR detection function
    pcen_snr = np.max(pcen,axis=0) - np.min(pcen,axis=0)
    pcen_snr = lb.power_to_db(pcen_snr / np.median(pcen_snr))
    if normalized:
        pcen_snr = pcen_snr / np.max(pcen_snr)
        
    # 4. Apply median filtering.
    if medfilt_time_constant is not None:
        medfilt_hops = medfilt_time_constant * sr / hop_length
        kernel_size = max(1, 1 + 2 * round(medfilt_hops - 0.5))
        pcen_snr = scipy.signal.medfilt(pcen_snr, kernel_size=kernel_size)
    
    # 5. Extract active segments.
    activity, start, end = threshold_activity(
        pcen_snr, peak_threshold, activity_threshold)
    
    # 6. Convert indices to seconds.
    start_times = np.round(np.array(start) * hop_length / sr, 3)
    end_times = np.round(np.array(end) * hop_length / sr, 3)
    
    return start_times, end_times

def threshold_activity(x, Tp, Ta):
    locs = signal.find_peaks(x,height = Tp)[0]
    y = (x > Ta) * 1
    act = np.diff(y)
    u = np.where(act == 1)[0]
    d = np.where(act == -1)[0]
    signal_length = len(x)
    
    if d[0] < u[0]:
        u = np.insert(u, 0, 0)
        
    if d[-1] < u[-1]:
        d = np.append(d, signal_length-1)
        
    starts = []
    ends = []
    
    activity = np.zeros(signal_length,)
    
    for candidate_up, candidate_down in zip(u, d):
        candidate_segment = range(candidate_up, candidate_down)
        peaks_in_segment = [x in candidate_segment for x in locs]
        is_valid_candidate = np.any(peaks_in_segment)
        if is_valid_candidate:
            starts.append(candidate_up)
            ends.append(candidate_down)
            activity[candidate_segment] = 1.0
            
    starts = np.array(starts)
    ends = np.array(ends)
    return activity, starts, ends

In [ ]:
# t-3 Get the stimuli in
stimuli_set = sio.loadmat('stimuli.mat')
stimuli = stimuli_set['audio_sti']

# (use the codes from above section to get imitations in again) test_1 to test_6
#test = sio.loadmat('test_1.mat')
#audio = test['audio_1']
#num_row = int(np.shape(audio)[0])



In [ ]:
# use detection_activity to get the number of syllables from all stimuli

# Since subject's data is distributed in several mat files
# we need to keep a count of how many is used eventually
total_subject = 0
each_number_events = []

for i in tqdm(range(100)):
    temp_sub = stimuli[i,0]
    temp_spe = stimuli[i,1]
    rdata = stimuli[i,2:-1]
    rdata_nz = np.nonzero(rdata)[0][-1]
    fdata = rdata[0:rdata_nz]
    
    if np.mean(lb.feature.spectral_flatness(fdata)) > 0.35:
        number_events.append(0)
    
    else:
        start, end = detect_activity(fdata,sr)
        activity_count = len(start)
        each_number_events.append(activity_count)
        
all_number_events = np.array(each_number_events)

In [ ]:
# t-4 use detection_activity to get the number of syllables from all imitations
sr = 44100

for j in range(int(num_row/100)):
    
    each_number_events = []

    for i in tqdm(range(100)):

        # Use this part for imitations
        temp_sub = audio[100*j+i,0]
        temp_spe = audio[100*j+i,1]
        rdata = audio[100*j+i,2:-1]
        rdata_nz = np.nonzero(rdata)[0][-1]
        fdata = rdata[0:rdata_nz]
 
        if np.mean(lb.feature.spectral_flatness(fdata)) > 0.35:
            number_events.append(0)
    
        else:
            start, end = detect_activity(fdata,sr)
            activity_count = len(start)
            each_number_events.append(activity_count)
    
    # counting the number of subject
    total_subject += 1        
 
    each_number_events = np.array(each_number_events)
    all_number_events = np.vstack((all_number_events, each_number_events))

In [ ]:
print(np.shape(each_number_events))
print(total_subject)

In [ ]:
# Calculate the ratio between the number of syllables in stimuli and imitations for each subject
for i in range(1,total_subject+1):
    if i == 1:
        data_ratio = np.round(np.array(all_number_events[i,:] / all_number_events[0,:]),decimals=1)
        print(np.shape(data_ratio))
    else:
        data_ratio = np.vstack((data_ratio,np.round(np.array(all_number_events[i,:] / all_number_events[0,:]),decimals = 1)))
        print(np.shape(data_ratio))        
        

In [ ]:
print(data_ratio)

In [ ]:
# Organize the species-wise ratio
for species in range(10):
    for i in range (total_subject):
        if i == 0:
            temp_data = data_ratio[i,species*10 : (species+1)*10-1]
        else:
            temp_data = np.hstack((temp_data,data_ratio[i,species*10 : (species+1)*10-1]))
            
    if species == 0:
        species_data = temp_data
    else:
        species_data = np.vstack((species_data,temp_data))
        

In [ ]:
# Density plot for each subject
from scipy.stats import gaussian_kde
for i in range(total_subject):
    if i == 0:
        density = gaussian_kde(data_ratio[i,:])
        xs = np.linspace(0,4,100)
        density.covariance_factor = lambda : .25
        density._compute_covariance()
        plt.plot(xs,density(xs))
    else:
        density = gaussian_kde(data_ratio[i,:])
        density.covariance_factor = lambda : .25
        density._compute_covariance()
        plt.plot(xs,density(xs))
    
plt.show()

In [ ]:
# Density plot for each species
for i in range(10):
    if i == 0:
        species_density = gaussian_kde(species_data[i,:])
        xs = np.linspace(0,4,100)
        species_density.covariance_factor = lambda : .25
        species_density._compute_covariance()
        plt.plot(xs,species_density(xs))
    else:
        species_density = gaussian_kde(species_data[i,:])
        species_density.covariance_factor = lambda : .25
        species_density._compute_covariance()
        plt.plot(xs,species_density(xs))
    
plt.show()